# 0. Konfiguration der Arbeitsumgebung
**Hinweis zur Ausführung:**

Bevor Sie mit dem Ausfüllen des Protokolls beginnen, stellt diese Zelle sicher, dass alle notwendigen Software-Bibliotheken installiert und verfügbar sind. Diese sogenannten „Pakete“ (z. B. `ipywidgets` für die interaktiven Eingabefelder) sind erforderlich, damit das Notebook korrekt funktioniert.

**Was in dieser Zelle passiert:**
- Es wird geprüft, ob alle benötigten Pakete bereits in Ihrer Jupyter-Umgebung installiert sind.
- Falls ein Paket fehlt, wird es automatisch installiert.
- Dies funktioniert unabhängig davon, ob Sie mit Anaconda (Windows) oder venv (macOS/Linux) arbeiten.

**Was Sie tun müssen:**
- Führen Sie diese Zelle **einmalig aus**, bevor Sie mit dem Protokoll arbeiten.
- Es ist **keine manuelle Installation** in der Konsole notwendig.
- Falls eine Installation erfolgt, kann es kurz dauern – warten Sie bitte, bis die Zelle vollständig durchgelaufen ist.

**Hinweis:** Falls Sie das Notebook ohne Internetverbindung verwenden, können fehlende Pakete nicht nachgeladen werden.

**Führen Sie jetzt die folgende Zelle aus:**

In [ ]:
import sys
import subprocess

# Benötigte Pakete
pakete = ["numpy", "pandas", "matplotlib", "tabulate", "prettytable", "ipywidgets", "requests", "coolprop"]

installiert = 0
fehler = []

# Pakete prüfen und installieren
for paket in pakete:
    try:
        __import__(paket)
    except ImportError:
        try:
            print(f"Installiere {paket}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", paket], 
                                 stdout=subprocess.DEVNULL)
            installiert += 1
        except:
            fehler.append(paket)

# Pakete importieren
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import ipywidgets as widgets
import requests, json, os
from CoolProp.CoolProp import PropsSI

# Status anzeigen
if fehler:
    print(f"FEHLER bei: {', '.join(fehler)}")
    print("-> Bitte manuell installieren: pip install <paketname>")
elif installiert > 0:
    print(f"{installiert} Pakete erfolgreich installiert!")
else:
    print("Alle Pakete bereits vorhanden!")

print("Setup abgeschlossen - bereit für das Praktikum!")

In [ ]:


# --- Konfiguration ---
VERSUCH = "Windkraft"  # ← hier den Versuchsnamen ändern
FILENAME = f"protokoll_{VERSUCH}.json"
FIELD_WIDTH = '500px'
LABEL_WIDTH = '160px'
LOGO_URL = "https://thga.sciebo.de/s/PMFfmxdyEcrEr3P/download"

# --- Felder und Standarddaten ---
fields = {
    "Name": "",
    "Matrikelnummer": "",
    "Gruppe": "",
    "Versuchstag": "",
    "Praktikumsleitung": ""
}

# --- Daten aus Datei laden ---
if os.path.exists(FILENAME):
    with open(FILENAME, "r") as f:
        fields.update(json.load(f))

# --- Logo laden ---
try:
    resp = requests.get(LOGO_URL)
    logo = widgets.Image(
        value=resp.content, format='png',
        layout=widgets.Layout(width='200px', height='auto')
    ) if resp.status_code == 200 else widgets.Label("Logo nicht verfügbar.")
except:
    logo = widgets.Label("Logo konnte nicht geladen werden.")

# --- Überschriftsteil ---
headings = widgets.VBox([
    widgets.HTML(f'<div style="font-size: 20pt; font-weight: bold;">Protokoll zum Versuch: {VERSUCH.capitalize()}</div>'),
    widgets.HTML(f'<div style="font-size: 14pt;"><strong>Praktikum:</strong> Regenerative Energien II</div>'),
    widgets.HTML(f'<div style="font-size: 14pt;"><strong>Semester:</strong> Wintersemester 2025/26</div>')
], layout=widgets.Layout(align_items='flex-start', gap='2px'))

header = widgets.HBox([headings, logo],
                      layout=widgets.Layout(justify_content='space-between', align_items='flex-start'))

# --- Formularfelder anlegen ---
widgets_dict = {
    key: widgets.Text(value=fields[key], layout=widgets.Layout(width=FIELD_WIDTH))
    for key in fields
}

def labeled_row(label, widget):
    return widgets.HBox([
        widgets.Label(label + ":", layout=widgets.Layout(width=LABEL_WIDTH)),
        widget
    ])

form = widgets.VBox([labeled_row(k, widgets_dict[k]) for k in fields])

# --- Speichern ---
def save_to_json(_):
    data = {k: w.value for k, w in widgets_dict.items()}
    with open(FILENAME, "w") as f:
        json.dump(data, f)
    print(f"Daten gespeichert in '{FILENAME}'.")

save_button = widgets.Button(description="Speichern", button_style='success')
save_button.on_click(save_to_json)

# --- Anzeige ---
display(header, widgets.HTML("<hr>"), form, save_button)

# 1. Einleitung
**Hinweis zur Bearbeitung:**

Bevor Sie mit dem Versuch beginnen, formulieren Sie bitte eine kurze Einleitung. Beschreiben Sie in eigenen Worten die grundlegenden physikalischen Prinzipien der Windenergienutzung und das Wissen, das benötigt wird, um dieses Protokoll nachvollziehen zu können.


 **Denkanstöße für Ihre Einleitung:**
- Wie entsteht Wind und welche globalen sowie lokalen Faktoren beeinflussen ihn?
- Welche physikalischen Größen sind für die Umwandlung von Windenergie in elektrische Energie relevant?
- Wie ist eine moderne Windkraftanlage aufgebaut und wie funktioniert sie?
- Was unterscheidet Anlagen mit horizontaler und vertikaler Drehachse?
- Welche Rolle spielt der Auftrieb an Rotorblättern bei der Energieerzeugung?
- Welche Bedeutung hat die Windenergie in der Energiewende?
- In welchen geografischen Regionen ist Windenergie besonders effizient nutzbar?
- Welche praktischen Anwendungen und aktuellen Entwicklungen der Windkraft kennen Sie?

 **Tragen Sie Ihre eigene Einleitung im folgenden Feld ein:**

# 2. Durchführung und Auswertung

## 2.1 Versuch 1: U-I-Kennlinie und Leistung verschiedener Rotorbauarten und Blattformen

In diesem Versuch werden verschiedene Rotoren (3-Blatt, 4-Blatt und Savonius) verwendet, um das Verhalten bei unterschiedlichen Rotorbauarten zu untersuchen. Ziel ist es, die Auswirkungen auf Stromstärke $( I )$, Spannung $( U )$, Leistung $( P )$ und Drehzahl $( D )$ zu erfassen.

Die Rotoren werden durch ein Winderzeugermodul mit einstellbarer Netzspannung (9 V) angetrieben. Die Messungen erfolgen mit einem AV-Modul und einem Drehzahlmessgerät.

---

**Aufbau, Geräte & Komponenten:**
- Grundeinheit  
- Winderzeugermodul  
- Windgeneratormodul (3- und 4-Flügler, optimiertes Profil)  
- Savoniusrotormodul  
- Potentiometermodul  
- AV-Modul  
- Drehzahlmessgerät  
- Power-Modul (9 V)

---

**Schaltplan:**

![Schaltplan Versuch1.1](Data/Schaltplan_Wind_Versuch_1.png "Schaltplan")

---

**Durchführung**

1. Bauen Sie den Versuchsplan mit dem 3-Blatt-Rotor (optimiertes Profil) auf.
   1. Stellen Sie eine Netzspannung von $U_\text{Netz} = 9\ \text{V}$ ein.
   2.  Messen Sie die Spannung $U$, Stromstärke $I$ und Drehzahl $d$ bei verschiedenen Potentiometerstellungen. Beginnen Sie bei der höchsten Spannung und verringern Sie diese in Schritten von ca. $0{,}2\ \text{V}$.
2. Wiederholen Sie den Versuch mit dem 4-Blatt-Rotor (optimiertes Profil).
3. Wiederholen Sie den Versuch mit dem Savoniusrotor. Verringern Sie die Spannung diesmal in Schritten von ca. $0{,}1\ \text{V}$.

---

**Messgrößen:**  
- Die elektrische Spannung $U$ in $\text{[V]}$  
- Die Stromstärke $I$ in $\text{[mA]}$  
- Die Leistung $P = U \cdot I$ in $\text{[mW]}$  
- Die Drehzahl $D$ in $\text{[min}^{-1}\text{]}$  

---

**Messdaten:**

Notieren Sie hier Ihre Messdaten in einfachen Listen. Nutzen Sie dafür eindeutige Bezeichnungen, ähnlich wie im folgenden Beispiel:

In [ ]:
# DATENERFASSUNG - NUR MESSWERTE EINTRAGEN!

# Messwerte für 3-Blatt-Rotor (optimiertes Profil)
u_3blatt = [3.74, 3.54, 3.33, 3.13, 2.92, 2.73, 2.51, 2.33, 2.12, 1.90, 1.71, 1.53, 1.32, 1.10, 0.855, 0.738]  # Spannung in V
i_3blatt = [4.1, 10.5, 20.2, 28.4, 37.2, 45.0, 53.4, 60.4, 68.6, 76.2, 82.6, 88.6, 94.9, 99.3, 101.7, 99.3]     # Strom in mA
D_3blatt = [4791, 4678, 4537, 4422, 4297, 4175, 4029, 3900, 3766, 3581, 3465, 3312, 3110, 2923, 2628, 2354]     # Drehzahl in 1/min

# Messwerte für 4-Blatt-Rotor (optimiertes Profil) 
u_4blatt = []  # Spannung in V
i_4blatt = []  # Strom in mA
D_4blatt = []      # Drehzahl in 1/min

# Messwerte für Savonius-Rotor
u_savonius = []  # Spannung in V
i_savonius = []         # Strom in mA
D_savonius = []           # Drehzahl in 1/min

print("Daten erfolgreich geladen!")
print("Verfügbare Datensätze:")
print(f"   - 3-Blatt-Rotor, optimiert:  {len(u_3blatt)} Messpunkte")
print(f"   - 4-Blatt-Rotor:  {len(u_4blatt)} Messpunkte")
print(f"   - Savonius-Rotor: {len(u_savonius)} Messpunkte")
print("Bereit für die Auswertung in den nächsten Zellen!")

Aus diesen Messwerten werden nun Tabellen mit `pandas` `DataFrames` erstellt.
Wichtige Hinweise:

- Die Messwerte müssen nicht nochmal eingegeben werden, solange die Zelle mit den Listen zuvor ausgeführt wurde
- Python beginnt die Indexierung bei [0] - der erste Wert hat Index 0, der zweite Index 1, usw.
- DataFrames ermöglichen professionelle Tabellendarstellung und einfache Berechnungen

Im folgenden Code werden alle Listen aus Messdaten in `DataFrames`umgewandelt:

In [ ]:
# DATAFRAMES AUS DEN LISTEN ERSTELLEN  

# 3-Blatt-Rotor, optimiert
df_3blatt = pd.DataFrame({
    "U [V]": u_3blatt,
    "I [mA]": i_3blatt,
    "D [1/min]": D_3blatt
})

# 4-Blatt-Rotor
df_4blatt = pd.DataFrame({
    "U [V]": u_4blatt,
    "I [mA]": i_4blatt,
    "D [1/min]": D_4blatt
})

# Savonius-Rotor
df_savonius = pd.DataFrame({
    "U [V]": u_savonius,
    "I [mA]": i_savonius,
    "D [1/min]": D_savonius
})

# Indizes als "Messpunkte" umbenennen (Zählung bei 1 starten)
for df in [df_3blatt, df_4blatt, df_savonius]:
    df.index = [f"Messpunkt {i+1}" for i in range(len(df))]

print("DataFrames erfolgreich erstellt:")
print(f"- df_3blatt: {len(df_3blatt)} Messpunkte")
print(f"- df_4blatt: {len(df_4blatt)} Messpunkte") 
print(f"- df_savonius: {len(df_savonius)} Messpunkte")

In [ ]:
# MESSDATENTABELLEN ANZEIGEN

print("Tabelle 1: Messdaten – 3-Blatt-Rotor (optimiertes Profil)")
display(df_3blatt.round(2).style.format(precision=2))

print("\nTabelle 2: Messdaten – 4-Blatt-Rotor")
display(df_4blatt.round(2).style.format(precision=2))

print("\nTabelle 4: Messdaten – Savonius-Rotor")
display(df_savonius.round(2).style.format(precision=2))

### Beobachtungen während des Versuchs
Beschreiben Sie hier **in eigenen Worten** den Aufbau, die Durchführung und Ihre Beobachtungen:
- Welche Messgrößen haben Sie bei den verschiedenen Rotortypen aufgenommen?  
- Wie haben sich Spannung und Strom bei den verschiedenen Rotorbauarten verändert?  
- Welche Unterschiede konnten Sie zwischen 3-Blatt-, 4-Blatt- und Savonius-Rotor feststellen?  
- Gab es besondere Auffälligkeiten bei einem der Rotoren?

> _Hinweis: Denken Sie an die Schritte „Aufbau, Messung und Beobachtung“._

---

**Im folgenden Feld Ihre eigene Beschreibung und Beobachtungen eintragen:**

### Auswertung

1. Berechnen Sie die Leistung des Generators für jeden Messpunkt.
2. Zeichnen Sie die jeweiligen U-I-Kennlinien. Erstellen Sie einen Vergleich für  
   a. die verschiedenen Flügelanzahl (3 vs. 4 Flügel)  
   b. die unterschiedlichen Rotorbauarten (3 Flügel, optimiert vs. Savonius-Rotor).
3. Zeichnen Sie nun die jeweiligen D-P-Diagramme und vergleichen Sie diese wie in 2. a–b.  
   Bestimmen Sie jeweils den Punkt maximaler Leistung.  
   a. Mit welcher Flügelanzahl kann die höchste Leistung erzielt werden?  
      Warum werden große Windkraftanlagen nahezu ausschließlich mit 3-Flügel-Rotoren gebaut?  
   b. Überlegen und begründen Sie, welche Windrotorbauart (Flügel oder Savonius) für den Betrieb in den folgenden Umgebungen am besten geeignet ist:  
      - Hohe Windgeschwindigkeit  
      - Niedrige Windgeschwindigkeit  
      - Stark wechselnde Windgeschwindigkeit

---

### Schritt 1: Leistungsberechnung

**Grundlagen:**
Die elektrische Leistung $P$ eines Generators berechnet sich aus dem Produkt von Spannung $U$ und Stromstärke $I$:

$$P = U \cdot I$$

Da unsere Messgrößen
- Spannung $U$ in [V] (Volt)
- Stromstärke $I$ in [mA] (Milliampere)

sind, ergibt sich die Leistung in [mW] (Milliwatt).

Die Leistung zeigt, wie effektiv ein Rotor Windenergie in elektrische Energie umwandelt. Ein Vergleich der Leistungswerte ermöglicht es, die verschiedenen Rotorbauarten zu bewerten.

**Was passiert im folgenden Code:**
- Für jeden Rotor wird dem `DataFrame`eine neue Spalte "P [mW]" hinzugefügt und in dieser die Leistung berechnet
- Die Berechnung erfolgt automatisch für alle Messpunkte gleichzeitig
- Die erweiterten DataFrames werden zur Kontrolle angezeigt

In [ ]:
# SCHRITT 1: LEISTUNGSBERECHNUNG FÜR ALLE ROTOREN

# Leistung P = U × I für jeden Rotor berechnen
df_3blatt["P [mW]"] = df_3blatt["U [V]"] * df_3blatt["I [mA]"]


print("Leistung erfolgreich berechnet!")

In [ ]:
# TABELLEN DER MESSDATEN MIT BERECHNETEN LEISTUNGEN ERSTELLEN

print("Tabelle 5: Messdaten und berechnete Leistung – 3-Blatt-Rotor, optimiert")
display(df_3blatt.round(2).style.format(precision=2))

print("Tabelle 6: Messdaten und berechnete Leistung – 4-Blatt-Rotor")
display(df_4blatt.round(2).style.format(precision=2))

print("Tabelle 8: Messdaten und berechnete Leistung – Savonius-Rotor")
display(df_savonius.round(2).style.format(precision=2))

### Schritt 2: U-I-Kennlinien

**Grundlagen:**
Die U-I-Kennlinie zeigt das Verhältnis zwischen Spannung $(U)$ und Stromstärke $(I)$ eines Generators. Sie ist charakteristisch für jede Rotorbauart und hilft beim Vergleich der elektrischen Eigenschaften.

**Was zeigen U-I-Kennlinien:**
- Wie sich die Spannung bei steigender Stromstärke verhält
- Unterschiede zwischen verschiedenen Rotortypen
- Elektrische Charakteristika der Generatoren

In [ ]:
# SCHRITT 2a: U-I-KENNLINIEN - VIER SEPARATE DIAGRAMME

# Vier Diagramme in 2x2 Anordnung erstellen
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# ============================================================================
# DIAGRAMM 1: 3-BLATT-ROTOR (OPTIMIERT)
# ============================================================================

ax1.plot(df_3blatt["U [V]"], df_3blatt["I [mA]"], "o", label="3-Blatt-Rotor (optimiert)", 
         markersize=6, color="blue")

ax1.set_xlabel("Spannung U [V]")
ax1.set_ylabel("Stromstärke I [mA]")
ax1.set_title("U-I-Kennlinie: 3-Blatt-Rotor (optimiert)")
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0)
ax1.set_ylim(0)
ax1.legend()

# ============================================================================
# DIAGRAMM 2: 4-BLATT-ROTOR
# ============================================================================

ax2.plot(df_4blatt["U [V]"], df_4blatt["I [mA]"], "s", label="4-Blatt-Rotor", 
         markersize=6, color="green")

ax2.set_xlabel("Spannung U [V]")
ax2.set_ylabel("Stromstärke I [mA]")
ax2.set_title("U-I-Kennlinie: 4-Blatt-Rotor")
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0)
ax2.set_ylim(0)
ax2.legend()

# ============================================================================
# DIAGRAMM 3: SAVONIUS-ROTOR
# ============================================================================

ax3.plot(df_savonius["U [V]"], df_savonius["I [mA]"], "d", label="Savonius-Rotor", 
         markersize=6, color="red")

ax3.set_xlabel("Stromstärke I [mA]")
ax3.set_ylabel("Spannung U [V]")
ax3.set_title("U-I-Kennlinie: Savonius-Rotor")
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0)
ax3.set_ylim(0)
ax3.legend()

# Layout optimieren und anzeigen
plt.tight_layout()
plt.show()

In [ ]:
# SCHRITT 2b: VERGLEICH DER U-I-KENNLINIEN

# Drei Vergleichsdiagramme nebeneinander
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# ============================================================================
# VERGLEICH 1: 3-BLATT OPTIMIERT vs 4-BLATT
# ============================================================================

ax1.plot(df_3blatt["U [V]"], df_3blatt["I [mA]"], "o", label="3-Blatt-Rotor (optimiert)", 
         markersize=6, color="blue")
ax1.plot(df_4blatt["U [V]"], df_4blatt["I [mA]"], "s", label="4-Blatt-Rotor", 
         markersize=6, color="green")

ax1.set_xlabel("Spannung U [V]")
ax1.set_ylabel("Stromstärke I [mA]")
ax1.set_title("Vergleich: Blätteranzahl\n3-Blatt vs 4-Blatt")
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0)
ax1.set_ylim(0)
ax1.legend()

# ============================================================================
# VERGLEICH 2: 3-BLATT OPTIMIERT vs SAVONIUS
# ============================================================================


plt.tight_layout()
plt.show()

### Schritt 3: D-P-Diagramme (Drehzahl-Leistung)

**Grundlagen**
Das D-P-Diagramm zeigt den Zusammenhang zwischen Drehzahl ($D$) und elektrischer Leistung ($P$). Es ist besonders wichtig, da aus diesem der **Punkt maximaler Leistung (MPP - Maximum Power Point)** für jeden Rotor abgeleitet werden kann.

**Wichtig: Der MPP ist nicht der von Ihnen höchste gemessener Wert, sondern liegt in der Regel zwischen Ihren Messwerten. Sie müssen den MPP also rechnerisch bestimmen!**

**Warum D-P-Diagramme wichtig sind:**
- Sie zeigen bei welcher Drehzahl die höchste Leistung erreicht wird
- Verschiedene Rotortypen haben unterschiedliche Leistungscharakteristika
- Basis für die Bewertung der Effizienz verschiedener Bauarten

In [ ]:
# SCHRITT 3a: D-P-DIAGRAMME - VERGLEICHE

# Drei Vergleichsdiagramme nebeneinander
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# ============================================================================
# VERGLEICH 1: 3-BLATT OPTIMIERT vs 4-BLATT
# ============================================================================

ax1.plot(df_3blatt["D [1/min]"], df_3blatt["P [mW]"], "o", label="3-Blatt-Rotor (optimiert)", 
         markersize=6, color="blue")
ax1.plot(df_4blatt["D [1/min]"], df_4blatt["P [mW]"], "s", label="4-Blatt-Rotor", 
         markersize=6, color="green")

ax1.set_xlabel("Drehzahl D [1/min]")
ax1.set_ylabel("Leistung P [mW]")
ax1.set_title("Vergleich: Blätteranzahl\n3-Blatt vs 4-Blatt")
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0)
ax1.set_ylim(0)
ax1.legend()

# ============================================================================
# VERGLEICH 3: 3-BLATT OPTIMIERT vs SAVONIUS
# ============================================================================



# Layout optimieren
plt.tight_layout()
plt.show()



In [ ]:
# SCHRITT 3b: MAXIMUM POWER POINT (MPP) BERECHNEN - KUBISCHE REGRESSION

# Fit einer kubischen Regressionskurve
reg_3blatt = np.poly1d(np.polyfit(df_3blatt["D [1/min]"], df_3blatt["P [mW]"], 3))


# Berechnung des Bestimmtheitsmaßes einer kubischen Funktion
regression_kubisch_3blatt = np.poly1d(np.polyfit(df_3blatt["D [1/min]"], df_3blatt["P [mW]"], 3))


leistung_pred_kubisch_3blatt = regression_kubisch_3blatt(df_3blatt["D [1/min]"])


SSR_kubisch_3blatt = np.sum((df_3blatt["P [mW]"] - leistung_pred_kubisch_3blatt) ** 2)


SST_kubisch_3blatt = np.sum((df_3blatt["P [mW]"] - np.mean(df_3blatt["P [mW]"])) ** 2)


R_squared_kubisch_3blatt = 1 - (SSR_kubisch_3blatt / SST_kubisch_3blatt)


print(f"Bestimmtheitsmaß (R²): {R_squared_kubisch_3blatt:.4f}", "für 3-Blatt (optimiert)")

print(" ")

# Bestimmung des MPP
regression_kubisch_3blatt = np.poly1d(np.polyfit(df_3blatt["D [1/min]"], df_3blatt["P [mW]"], 3))


ableitung_3blatt=np.polyder(regression_kubisch_3blatt)


zweite_ableitung_3blatt=np.polyder(ableitung_3blatt)


nullstellen_3blatt=np.roots(ableitung_3blatt)


print("3-Blatt (optimiert):")
for x in nullstellen_3blatt:
    wert_zweite_ableitung_3blatt = zweite_ableitung_3blatt(x)
    if wert_zweite_ableitung_3blatt > 0:
        print(f"x = {x:.2f} ist ein Minimum (f""(x) = {wert_zweite_ableitung_3blatt:.4f})")
    elif wert_zweite_ableitung_3blatt < 0:
        print(f"x = {x:.2f} ist ein Maximum (f""(x) = {wert_zweite_ableitung_3blatt:.4f})")
    else:
        print(f"x = {x:.2f} könnte ein Sattelpunkt sein (f""(x) = {wert_zweite_ableitung_3blatt:.4f})")

print(" ")


MPP_3blatt=regression_kubisch_3blatt(nullstellen_3blatt[0])

print(" ")
print("Der MPP mit 3-Blatt (optimiert) liegt bei", f"{MPP_3blatt:.2f}", "mW")

print(" ")

# Zwei Diagramme erstellen
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))


# Bereiche für glatte Regressionskurven (angepasst an tatsächliche Messbereiche)
reg_bereich_3blatt = np.linspace(min(df_3blatt["D [1/min]"]), max(df_3blatt["D [1/min]"]), 100)


# ============================================================================
# DIAGRAMM 1: 3-Blatt optimiert vs 4-Blatt
# ============================================================================

ax1.plot(df_3blatt["D [1/min]"], df_3blatt["P [mW]"], "o", label="3-Blatt (optimiert) Messdaten", color="blue")
ax1.plot(reg_bereich_3blatt, reg_3blatt(reg_bereich_3blatt), "--", color="lightblue", label="Regression 3-Blatt optimiert")

# MPP markieren
ax1.plot(nullstellen_3blatt[0], MPP_3blatt, "o", label="MPP 3-Blatt", color="darkblue", markersize=8)


ax1.set_xlabel("Drehzahl D [1/min]")
ax1.set_ylabel("Leistung P [mW]")
ax1.set_title("D-P-Diagramm: 3-Blatt vs 4-Blatt (mit MPP)")
ax1.grid()
ax1.set_xlim(0)
ax1.set_ylim(0)
ax1.legend()

# ============================================================================
# DIAGRAMM 2: 3-Blatt optimiert vs Savonius
# ============================================================================

ax2.plot(df_3blatt["D [1/min]"], df_3blatt["P [mW]"], "o", label="3-Blatt (optimiert) Messdaten", color="blue")
ax2.plot(reg_bereich_3blatt, reg_3blatt(reg_bereich_3blatt), "--", color="lightblue", label="Regression 3-Blatt")


# MPP markieren
ax2.plot(nullstellen_3blatt[0], MPP_3blatt, "o", label="MPP 3-Blatt", color="darkblue", markersize=8)


ax2.set_xlabel("Drehzahl D [1/min]")
ax2.set_ylabel("Leistung P [mW]")
ax2.set_title("D-P-Diagramm: 3-Blatt vs Savonius (mit MPP)")
ax2.grid()
ax2.set_xlim(0)
ax2.set_ylim(0)
ax2.legend()

plt.tight_layout()
plt.show()

print("")

### Auswertung und Diskussion der Ergebnisse

#### a) Flügelanzahl und Leistungsvergleich

**Messergebnis:** 3-Blatt- und 4-Blatt-Läufer mit optimierten Profilen erreichen ähnliche Maximalleistungen.

**Warum verwenden große Windkraftanlagen 3-Blatt-Läufer?**

Technisch-wirtschaftliches Optimum: Drei Blätter gewährleisten ausreichende Läufersymmetrie bei minimalem Materialaufwand. Die ungerade Blattzahl verhindert Resonanzeffekte zwischen Läufer- und Turmfrequenz. Der aerodynamische Wirkungsgrad erreicht bei drei Blättern einen Sättigungsbereich - zusätzliche Blätter bringen proportional geringere Leistungssteigerungen bei höheren Kosten.

#### b) Läuferbauarten für verschiedene Windverhältnisse

**Hohe Windgeschwindigkeit:** Auftriebsläufer (3-Blatt, optimiertes Profil) erreichen höchste spezifische Leistungen durch hohe Schnelllaufzahlen. Schutz vor Überlastung durch aktive Blattverstellung.

**Niedrige Windgeschwindigkeit:** Widerstandsläufer (Savonius-Typ) haben niedrigere Anlaufwindgeschwindigkeit (1-2 m/s) und verfügbares Anlaufdrehmoment bei geringen Drehzahlen. Auftriebsläufer benötigen 3-4 m/s Anfahrwindgeschwindigkeit. Richtungsunabhängigkeit eliminiert Windrichtungsnachführung.

**Stark wechselnde Windgeschwindigkeit:** Savonius-Läufer zeigen höhere Läuferträgheit (glättet Leistungsschwankungen), geringere strukturelle Beanspruchung durch einfachen Aufbau, und omnidirektionale Funktionalität ohne Leistungseinbußen bei Windrichtungsänderungen.

**Messbeschränkungen:** Die Laborversuche simulieren konstante Anströmbedingungen. Reale atmosphärische Turbulenz und variable Anströmwinkel können die Leistungscharakteristika erheblich verändern.

---

## 2.2 Versuch 2: Abhängigkeit der Leistung vom Anstellwinkel der Flügel

In diesem Versuch wird untersucht, wie sich die elektrische Leistung des Windkraftmodells in Abhängigkeit vom Anstellwinkel der Rotorblätter verändert. Dazu wird bei einem 3-Blatt-Rotor (optimiertes Profil) der Winkel der Flügel in mehreren Stufen variiert. Ziel ist es, den Einfluss des Anstellwinkels auf Spannung $(U)$, Stromstärke $(I)$ und die daraus berechnete Leistung $(P = U \cdot I)$ zu analysieren.

---

**Aufbau, Geräte & Komponenten:**
- Grundeinheit  
- Winderzeugermodul  
- Windgeneratormodul mit 3-Blatt-Rotor (optimiertes Profil, verstellbarer Anstellwinkel)  
- Potentiometermodul  
- AV-Modul  
- Drehzahlmessgerät  
- Power-Modul (9 V)  

---

**Schaltplan:**

![Schaltplan Versuch2](Data/Schaltplan_Wind_Versuch_2.png "Schaltplan")

---

**Durchführung**

1. Bauen Sie den Versuchsaufbau mit dem 3-Blatt-Rotor (optimiertes Profil) auf.
   1. Stellen Sie eine feste Netzspannung von $U_\text{Netz} = 9\ \text{V}$ ein.
   2. Stellen Sie den Anstellwinkel der Flügel nacheinander auf 15°, 30°, 45°, 60°, 75° und 90° ein.
   3. Messen Sie bei jedem Winkel die Spannung $U$, Stromstärke $I$ und die Drehzahl $D$.
2. Berechnen Sie für jeden Messpunkt die Leistung $P = U \cdot I$.

---

**Messgrößen:**  
- Die elektrische Spannung $U$ in $\text{[V]}$  
- Die Stromstärke $I$ in $\text{[mA]}$  
- Die Leistung $P = U \cdot I$ in $\text{[mW]}$  
- Die Drehzahl $D$ in $\text{[min}^{-1}\text{]}$  

---

**Messdaten:**

Notieren Sie hier Ihre Messdaten in einfachen Listen. Nutzen Sie dafür eindeutige Bezeichnungen, ähnlich wie im folgenden Beispiel:

In [ ]:
# DATENERFASSUNG - NUR MESSWERTE EINTRAGEN!

# Messwerte für 3-Blatt-Rotor, Anstellwinkel α = 20°
u_20grad = [3.67, 3.47, 3.26, 3.06, 2.86, 2.65, 2.46, 2.25, 2.06, 1.85, 1.62, 1.49]  # Spannung in V
i_20grad = [3.9, 12.7, 21.5, 29.7, 37.0, 45.9, 52.0, 58.9, 65.2, 71.7, 77.2, 77.2]   # Strom in mA
D_20grad = [4733, 4593, 4476, 4351, 4203, 4071, 3888, 3779, 3637, 3465, 3112, 3065]  # Drehzahl in 1/min

# Messwerte für 3-Blatt-Rotor, Anstellwinkel α = 25° (aus Versuch 1 übernehmen)
u_25grad = u_3blatt
i_25grad = i_3blatt
D_25grad = D_3blatt

# Messwerte für 3-Blatt-Rotor, Anstellwinkel α = 30°
u_30grad = [2.88, 2.68, 2.49, 2.28, 2.08, 1.85, 1.65, 1.47, 1.23, 1.05, 0.805, 0.671, 0.625]  # Spannung in V
i_30grad = [3.2, 13.1, 22.7, 32.0, 40.7, 50.8, 59.3, 66.9, 75.6, 82.2, 90.2, 93.3, 93.6]       # Strom in mA
D_30grad = [3706, 3593, 3496, 3351, 3228, 3116, 2967, 2881, 2710, 2567, 2375, 2242, 2188]     # Drehzahl in 1/min

# Messwerte für 3-Blatt-Rotor, Anstellwinkel α = 50°
u_50grad = [1.24, 1.04, 0.84, 0.615, 0.394, 0.351]  # Spannung in V
i_50grad = [1.8, 13.4, 24.7, 37.4, 48.7, 50.7]       # Strom in mA
D_50grad = [1570, 1485, 1395, 1297, 1176, 1146]      # Drehzahl in 1/min

print("Daten erfolgreich geladen!")
print("Verfügbare Datensätze für Anstellwinkel-Vergleich:")
print(f"   - α = 20°: {len(u_20grad)} Messpunkte")
print(f"   - α = 25°: {len(u_25grad)} Messpunkte")
print(f"   - α = 30°: {len(u_30grad)} Messpunkte")
print(f"   - α = 50°: {len(u_50grad)} Messpunkte")
print("Bereit für die Auswertung in den nächsten Zellen!")

Aus diesen Messwerten werden nun Tabellen mit `pandas` `DataFrames` erstellt.
Wichtige Hinweise:

- Die Messwerte müssen nicht nochmal eingegeben werden, solange die Zelle mit den Listen zuvor ausgeführt wurde
- Python beginnt die Indexierung bei [0] - der erste Wert hat Index 0, der zweite Index 1, usw.
- DataFrames ermöglichen professionelle Tabellendarstellung und einfache Berechnungen

Im folgenden Code werden alle Listen aus Messdaten in `DataFrames`umgewandelt:

In [ ]:
# DATAFRAMES AUS DEN LISTEN ERSTELLEN  

# 3-Blatt-Rotor, Anstellwinkel α = 20°
df_20grad = pd.DataFrame({
    "U [V]": ,
    "I [mA]": ,
    "D [1/min]": 
})

# 3-Blatt-Rotor, Anstellwinkel α = 25°
df_25grad = pd.DataFrame({
    "U [V]": ,
    "I [mA]": ,
    "D [1/min]": 
})

# 3-Blatt-Rotor, Anstellwinkel α = 30°
df_30grad = pd.DataFrame({
    "U [V]": ,
    "I [mA]": ,
    "D [1/min]": 
})

# 3-Blatt-Rotor, Anstellwinkel α = 50°
df_50grad = pd.DataFrame({
    "U [V]": ,
    "I [mA]": ,
    "D [1/min]": 
})

# Indizes als "Messpunkte" umbenennen (Zählung bei 1 starten)
for df in [df_20grad, df_25grad, df_30grad, df_50grad]:
    df.index = [f"Messpunkt {i+1}" for i in range(len(df))]

print("DataFrames erfolgreich erstellt:")
print(f"- df_20grad: {len(df_20grad)} Messpunkte")
print(f"- df_25grad: {len(df_25grad)} Messpunkte") 
print(f"- df_30grad: {len(df_30grad)} Messpunkte")
print(f"- df_50grad: {len(df_50grad)} Messpunkte")

In [ ]:
# MESSDATENTABELLEN ANZEIGEN
print("Tabelle 9: Messdaten – 3-Blatt-Rotor, Anstellwinkel α = 20°")
print(tabulate(df_, headers=df_.columns, tablefmt="fancy_grid", 
               floatfmt=".2f"))
print("\n")
print("Tabelle 10: Messdaten – 3-Blatt-Rotor, Anstellwinkel α = 25°")
print(tabulate(df_, headers=df_.columns, tablefmt="fancy_grid",
               floatfmt=".2f"))
print("\n")
print("Tabelle 11: Messdaten – 3-Blatt-Rotor, Anstellwinkel α = 30°")
print(tabulate(df_, headers=df_.columns, tablefmt="fancy_grid", 
               floatfmt=".2f"))
print("\n")
print("Tabelle 12: Messdaten – 3-Blatt-Rotor, Anstellwinkel α = 50°")
print(tabulate(df_, headers=df_.columns, tablefmt="fancy_grid",
               floatfmt=".2f"))

### Beobachtungen während des Versuchs
Beschreiben Sie hier **in eigenen Worten** den Aufbau, die Durchführung und Ihre Beobachtungen:

- Welche Anstellwinkel haben Sie bei dem 3-Blatt-Rotor untersucht und warum ist diese Variation wichtig?
- Wie haben sich Spannung, Strom und Drehzahl bei den verschiedenen Anstellwinkeln verändert?
- Bei welchem Anstellwinkel haben Sie die höchsten bzw. niedrigsten Leistungswerte gemessen?
- Welche praktischen Auswirkungen könnte die Verstellbarkeit des Anstellwinkels für reale Windkraftanlagen haben?
- Gab es bei einem bestimmten Anstellwinkel besondere Auffälligkeiten in der Drehzahl oder im Betriebsverhalten?

> *Hinweis: Denken Sie an die aerodynamischen Grundlagen - warum beeinflusst der Anstellwinkel die Leistung so stark?*

---

**Im folgenden Feld Ihre eigene Beschreibung und Beobachtungen eintragen:**

### Auswertung
1. Berechnen Sie die Leistung des Generators für jeden Messpunkt.
2. Zeichnen Sie die jeweiligen U-I-Kennlinien und die D-P-Diagramme und vergleichen Sie die verschiedenen Anstellwinkel miteinander.
3. Bestimmen Sie für jeden Anstellwinkel den Punkt maximaler Leistung (Maximum Power Point, MPP).
4. Welchen Einfluss hat der Anstellwinkel auf die maximal erzielte Leistung?  
   Wie kann das beobachtete Verhalten für den Betrieb realer Windkraftanlagen genutzt werden, um Schäden an der Anlage zu vermeiden?  
   Wie heißt die entsprechende Regelung?

### Schritt 1: Leistungsberechnung
**Grundlagen:**
Die elektrische Leistung $P$ eines Generators berechnet sich aus dem Produkt von Spannung $U$ und Stromstärke $I$:
$$P = U \cdot I$$
Da unsere Messgrößen
- Spannung $U$ in [V] (Volt)
- Stromstärke $I$ in [mA] (Milliampere)

sind, ergibt sich die Leistung in [mW] (Milliwatt).

Bei der Untersuchung verschiedener Anstellwinkel können wir durch die Leistungsberechnung erkennen, wie sich die aerodynamische Effizienz der Rotorblätter verändert. Der Anstellwinkel beeinflusst den Auftrieb und Widerstand der Blätter und damit direkt die übertragene mechanische Leistung auf den Generator.

**Was passiert im folgenden Code:**
- Für jeden Anstellwinkel wird dem `DataFrame` eine neue Spalte "P [mW]" hinzugefügt und in dieser die Leistung berechnet
- Die Berechnung erfolgt automatisch für alle Messpunkte gleichzeitig
- Die erweiterten DataFrames werden zur Kontrolle angezeigt

In [ ]:
# SCHRITT 1: LEISTUNGSBERECHNUNG FÜR ALLE ROTOREN

# Leistung P = U × I für jeden Rotor berechnen
df_["P [mW]"] = df_["U [V]"] * df_"I [mA]"]
df_["P [mW]"] = df_["U [V]"] * df_["I [mA]"]
df_["P [mW]"] = df_["U [V]"] * df_["I [mA]"]
df_["P [mW]"] = df_["U [V]"] * df_["I [mA]"]

print("Leistung erfolgreich berechnet!")

In [ ]:
# ERWEITERTE TABELLEN MIT LEISTUNG ANZEIGEN
print("Tabelle 13: Messdaten und berechnete Leistung – 3-Blatt-Rotor, Anstellwinkel α = 20°")
print(tabulate(df_, headers=df_.columns, tablefmt="fancy_grid", 
               floatfmt=".2f"))
print("\n")
print("Tabelle 14: Messdaten und berechnete Leistung – 3-Blatt-Rotor, Anstellwinkel α = 25°")
print(tabulate(df_, headers=df_.columns, tablefmt="fancy_grid",
               floatfmt=".2f"))
print("\n")
print("Tabelle 15: Messdaten und berechnete Leistung – 3-Blatt-Rotor, Anstellwinkel α = 30°")
print(tabulate(df_, headers=df_.columns, tablefmt="fancy_grid", 
               floatfmt=".2f"))
print("\n")
print("Tabelle 16: Messdaten und berechnete Leistung – 3-Blatt-Rotor, Anstellwinkel α = 50°")
print(tabulate(df_, headers=df_.columns, tablefmt="fancy_grid",
               floatfmt=".2f"))

### Schritt 2: U-I-Kennlinien

**Grundlagen:**
Die U-I-Kennlinie zeigt das Verhältnis zwischen Spannung $(U)$ und Stromstärke $(I)$ eines Generators. Sie ist charakteristisch für jede Rotorbauart und hilft beim Vergleich der elektrischen Eigenschaften.

**Was zeigen U-I-Kennlinien:**
- Wie sich die Spannung bei steigender Stromstärke verhält
- Unterschiede zwischen verschiedenen Rotortypen
- Elektrische Charakteristika der Generatoren

In [ ]:
# SCHRITT 2a: U-I-KENNLINIEN - VIER SEPARATE DIAGRAMME

# Vier Diagramme in 2x2 Anordnung erstellen
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# ============================================================================
# DIAGRAMM 1: 3-BLATT-ROTOR, Anstellwinkel α = 20°
# ============================================================================
ax1.plot(df_20grad["U [V]"], df_20grad["I [mA]"], "o", label="3-Blatt-Rotor, α = 20°", 
         markersize=6, color="blue")
ax1.set_xlabel("Spannung U [V]")
ax1.set_ylabel("Stromstärke I [mA]")
ax1.set_title("U-I-Kennlinie: 3-Blatt-Rotor, α = 20°")
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0)
ax1.set_ylim(0)
ax1.legend()

# ============================================================================
# DIAGRAMM 2: 3-BLATT-ROTOR, Anstellwinkel α = 25°
# ============================================================================
ax2.plot(df_25grad["U [V]"], df_25grad["I [mA]"], "s", label="3-Blatt-Rotor, α = 25°", 
         markersize=6, color="green")
# TODO: Ergänzen Sie xlabel, ylabel, title, grid, legend

# ============================================================================
# DIAGRAMM 3: 3-BLATT-ROTOR, Anstellwinkel α = 30°
# ============================================================================
# TODO: Vervollständigen Sie dieses Diagramm analog zu den oberen
# Verwenden Sie "^" als Marker und "orange" als Farbe

# ============================================================================
# DIAGRAMM 4: 3-BLATT-ROTOR, Anstellwinkel α = 50°
# ============================================================================  
# TODO: Vervollständigen Sie dieses Diagramm analog zu den oberen
# Verwenden Sie "d" als Marker und "red" als Farbe

# Layout optimieren und anzeigen
plt.tight_layout()
plt.show()

# ============================================================================
# ANALYSE DER U-I-KENNLINIEN
# ============================================================================
print("=" * 60)
print("ANALYSE DER U-I-KENNLINIEN")
print("=" * 60)
print("Spannungsbereiche:")
print(f"α = 20°: {min(df_20grad["U [V]"]):.2f} V bis {max(df_20grad["U [V]"]):.2f} V")
print(f"α = 25°: {min(df_25grad["U [V]"]):.2f} V bis {max(df_25grad["U [V]"]):.2f} V")
# TODO: Ergänzen Sie die Ausgabe für α = 30° und α = 50°

print()
print("Strombereiche:")
print(f"α = 20°: {min(df_20grad["I [mA]"]):.1f} mA bis {max(df_20grad["I [mA]"]):.1f} mA")
# TODO: Ergänzen Sie die Ausgabe für die anderen Anstellwinkel

print()
print("Charakteristische Unterschiede:")
print("- TODO: Analysieren Sie die Unterschiede zwischen den Anstellwinkeln")
print("- TODO: Welcher Anstellwinkel zeigt die besten Eigenschaften?")
print("- TODO: Wie wirkt sich der Anstellwinkel auf die Kennlinie aus?")
print("=" * 60)

In [ ]:
# SCHRITT 2b: VERGLEICH DER U-I-KENNLINIEN
# ============================================================================
# VERGLEICH: ALLE ANSTELLWINKEL
# ============================================================================
fig, ax1 = plt.subplots(1, 1, figsize=(10, 6))

ax1.plot(df_20grad["U [V]"], df_20grad["I [mA]"], "o", label="3-Blatt-Rotor, Anstellwinkel α = 20°", 
         markersize=6, color="blue")
# TODO: Vervollständigen Sie die anderen Plots
ax1.plot(df_25grad["U [V]"], df_25grad["I [mA]"], "s", label="3-Blatt-Rotor, Anstellwinkel α = 25°", 
         markersize=6, color="green")
# TODO: Ergänzen Sie df_30grad mit "^" als Marker und "orange" als Farbe
# TODO: Ergänzen Sie df_50grad mit "d" als Marker und "red" als Farbe

ax1.set_xlabel("Spannung U [V]")
ax1.set_ylabel("Stromstärke I [mA]")
ax1.set_title("Vergleich: Anstellwinkel von 20°, 25°, 30° und 50°")
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0)
ax1.set_ylim(0)
ax1.legend()

# Layout optimieren
plt.tight_layout()
plt.show()

# ============================================================================
# ANALYSE DER VERGLEICHE
# ============================================================================
print("=" * 70)
print("ANALYSE DER U-I-KENNLINIEN-VERGLEICHE")
print("=" * 70)
print("VERGLEICH - Anstellwinkel von 20°, 25°, 30° und 50°:")
print("- Spannungsbereich:")
print(f"  α = 20°: {min(df_20grad["U [V]"]):.2f} V bis {max(df_20grad["U [V]"]):.2f} V")
# TODO: Ergänzen Sie die Ausgabe für die anderen Anstellwinkel

print("- Maximaler Strom:")
print(f"  α = 20°: {max(df_20grad["I [mA]"]):.1f} mA")
# TODO: Ergänzen Sie die Ausgabe für die anderen Anstellwinkel

print()
print("SCHLUSSFOLGERUNGEN:")
print("- TODO: Welcher Anstellwinkel zeigt die besten Eigenschaften?")
print("- TODO: Wie wirkt sich der Anstellwinkel auf die Kennlinie aus?")
print("=" * 70)

### Schritt 3: D-P-Diagramme (Drehzahl-Leistung)

**Grundlagen**
Das D-P-Diagramm zeigt den Zusammenhang zwischen Drehzahl ($D$) und elektrischer Leistung ($P$). Es ist besonders wichtig, da aus diesem der **Punkt maximaler Leistung (MPP - Maximum Power Point)** für jeden Rotor abgeleitet werden kann.

**Wichtig: Der MPP ist nicht der von Ihnen höchste gemessener Wert, sondern liegt in der Regel zwischen Ihren Messwerten. Sie müssen den MPP also rechnerisch bestimmen!**

**Warum D-P-Diagramme wichtig sind:**
- Sie zeigen bei welcher Drehzahl die höchste Leistung erreicht wird
- Verschiedene Rotortypen haben unterschiedliche Leistungscharakteristika
- Basis für die Bewertung der Effizienz verschiedener Bauarten

In [ ]:
# SCHRITT 3a: D-P-DIAGRAMME - VIER SEPARATE DIAGRAMME
# Vier Diagramme in 2x2 Anordnung erstellen
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# ============================================================================
# DIAGRAMM 1: 3-BLATT-ROTOR, Anstellwinkel α = 20°
# ============================================================================
ax1.plot(df_20grad["D [1/min]"], df_20grad["P [mW]"], "o", label="3-Blatt-Rotor, α = 20°", 
         markersize=6, color="blue")
ax1.set_xlabel("Drehzahl D [1/min]")
ax1.set_ylabel("Leistung P [mW]")
ax1.set_title("D-P-Diagramm: 3-Blatt-Rotor, α = 20°")
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0)
ax1.set_ylim(0)
ax1.legend()

# ============================================================================
# DIAGRAMM 2: 3-BLATT-ROTOR, Anstellwinkel α = 25°
# ============================================================================
ax2.plot(df_25grad["D [1/min]"], df_25grad["P [mW]"], "s", label="3-Blatt-Rotor, α = 25°", 
         markersize=6, color="green")
# TODO: Ergänzen Sie xlabel, ylabel, title, grid, legend

# ============================================================================
# DIAGRAMM 3: 3-BLATT-ROTOR, Anstellwinkel α = 30°
# ============================================================================
# TODO: Vervollständigen Sie dieses Diagramm analog zu den oberen
# Verwenden Sie "^" als Marker und "orange" als Farbe

# ============================================================================
# DIAGRAMM 4: 3-BLATT-ROTOR, Anstellwinkel α = 50°
# ============================================================================  
# TODO: Vervollständigen Sie dieses Diagramm analog zu den oberen
# Verwenden Sie "d" als Marker und "red" als Farbe

# Layout optimieren und anzeigen
plt.tight_layout()
plt.show()

# ============================================================================
# ANALYSE DER D-P-DIAGRAMME
# ============================================================================
print("=" * 70)
print("ANALYSE DER D-P-DIAGRAMME")
print("=" * 70)
print("Drehzahlbereiche:")
print(f"α = 20°: {min(df_20grad["D [1/min]"]):.0f} 1/min bis {max(df_20grad["D [1/min]"]):.0f} 1/min")
# TODO: Ergänzen Sie die Ausgabe für die anderen Anstellwinkel

print()
print("Maximale Leistung:")
print(f"α = 20°: {max(df_20grad["P [mW]"]):.2f} mW")
# TODO: Ergänzen Sie die Ausgabe für die anderen Anstellwinkel

print()
print("Charakteristische Unterschiede:")
print("- TODO: Analysieren Sie die Unterschiede zwischen den Anstellwinkeln")
print("=" * 70)

In [ ]:
# SCHRITT 3b: MAXIMUM POWER POINT (MPP) BERECHNEN - KUBISCHE REGRESSION

# Fit einer kubischen Regressionskurve
reg_20grad = np.poly1d(np.polyfit(df_20grad["D [1/min]"], df_20grad["P [mW]"], 3))
reg_25grad = np.poly1d(np.polyfit(df_25grad["D [1/min]"], df_25grad["P [mW]"], 3))
reg_30grad = np.poly1d(np.polyfit(df_30grad["D [1/min]"], df_30grad["P [mW]"], 3))
reg_50grad = np.poly1d(np.polyfit(df_50grad["D [1/min]"], df_50grad["P [mW]"], 3))

# Berechnung des Bestimmtheitsmaßes einer kubischen Funktion
regression_kubisch_20grad = np.poly1d(np.polyfit(df_20grad["D [1/min]"], df_20grad["P [mW]"], 3))
regression_kubisch_25grad = np.poly1d(np.polyfit(df_25grad["D [1/min]"], df_25grad["P [mW]"], 3))
regression_kubisch_30grad = np.poly1d(np.polyfit(df_30grad["D [1/min]"], df_30grad["P [mW]"], 3))
regression_kubisch_50grad = np.poly1d(np.polyfit(df_50grad["D [1/min]"], df_50grad["P [mW]"], 3))

leistung_pred_kubisch_20grad = regression_kubisch_20grad(df_20grad["D [1/min]"])
leistung_pred_kubisch_25grad = regression_kubisch_25grad(df_25grad["D [1/min]"])
leistung_pred_kubisch_30grad = regression_kubisch_30grad(df_30grad["D [1/min]"])
leistung_pred_kubisch_50grad = regression_kubisch_50grad(df_50grad["D [1/min]"])

SSR_kubisch_20grad = np.sum((df_20grad["P [mW]"] - leistung_pred_kubisch_20grad) ** 2)
SSR_kubisch_25grad = np.sum((df_25grad["P [mW]"] - leistung_pred_kubisch_25grad) ** 2)
SSR_kubisch_30grad = np.sum((df_30grad["P [mW]"] - leistung_pred_kubisch_30grad) ** 2)
SSR_kubisch_50grad = np.sum((df_50grad["P [mW]"] - leistung_pred_kubisch_50grad) ** 2)

SST_kubisch_20grad = np.sum((df_20grad["P [mW]"] - np.mean(df_20grad["P [mW]"])) ** 2)
SST_kubisch_25grad = np.sum((df_25grad["P [mW]"] - np.mean(df_25grad["P [mW]"])) ** 2)
SST_kubisch_30grad = np.sum((df_30grad["P [mW]"] - np.mean(df_30grad["P [mW]"])) ** 2)
SST_kubisch_50grad = np.sum((df_50grad["P [mW]"] - np.mean(df_50grad["P [mW]"])) ** 2)

R_squared_kubisch_20grad = 1 - (SSR_kubisch_20grad / SST_kubisch_20grad)
R_squared_kubisch_25grad = 1 - (SSR_kubisch_25grad / SST_kubisch_25grad)
R_squared_kubisch_30grad = 1 - (SSR_kubisch_30grad / SST_kubisch_30grad)
R_squared_kubisch_50grad = 1 - (SSR_kubisch_50grad / SST_kubisch_50grad)

print(f"Bestimmtheitsmaß (R²): {R_squared_kubisch_20grad:.4f}", "für α = 20°")
print(f"Bestimmtheitsmaß (R²): {R_squared_kubisch_25grad:.4f}", "für α = 25°")
print(f"Bestimmtheitsmaß (R²): {R_squared_kubisch_30grad:.4f}", "für α = 30°")
print(f"Bestimmtheitsmaß (R²): {R_squared_kubisch_50grad:.4f}", "für α = 50°")
print(" ")

# Bestimmung des MPP
regression_kubisch_20grad = np.poly1d(np.polyfit(df_20grad["D [1/min]"], df_20grad["P [mW]"], 3))
regression_kubisch_25grad = np.poly1d(np.polyfit(df_25grad["D [1/min]"], df_25grad["P [mW]"], 3))
regression_kubisch_30grad = np.poly1d(np.polyfit(df_30grad["D [1/min]"], df_30grad["P [mW]"], 3))
regression_kubisch_50grad = np.poly1d(np.polyfit(df_50grad["D [1/min]"], df_50grad["P [mW]"], 3))

ableitung_20grad=np.polyder(regression_kubisch_20grad)
ableitung_25grad=np.polyder(regression_kubisch_25grad)
ableitung_30grad=np.polyder(regression_kubisch_30grad)
ableitung_50grad=np.polyder(regression_kubisch_50grad)

zweite_ableitung_20grad=np.polyder(ableitung_20grad)
zweite_ableitung_25grad=np.polyder(ableitung_25grad)
zweite_ableitung_30grad=np.polyder(ableitung_30grad)
zweite_ableitung_50grad=np.polyder(ableitung_50grad)

nullstellen_20grad=np.roots(ableitung_20grad)
nullstellen_25grad=np.roots(ableitung_25grad)
nullstellen_30grad=np.roots(ableitung_30grad)
nullstellen_50grad=np.roots(ableitung_50grad)

print("α = 20°:")
for x in nullstellen_20grad:
    wert_zweite_ableitung_20grad = zweite_ableitung_20grad(x)
    if wert_zweite_ableitung_20grad > 0:
        print(f"x = {x:.2f} ist ein Minimum (f""(x) = {wert_zweite_ableitung_20grad:.4f})")
    elif wert_zweite_ableitung_20grad < 0:
        print(f"x = {x:.2f} ist ein Maximum (f""(x) = {wert_zweite_ableitung_20grad:.4f})")
    else:
        print(f"x = {x:.2f} könnte ein Sattelpunkt sein (f""(x) = {wert_zweite_ableitung_20grad:.4f})")

print(" ")
print("α = 25°:")
for x in nullstellen_25grad:
    wert_zweite_ableitung_25grad = zweite_ableitung_25grad(x)
    if wert_zweite_ableitung_25grad > 0:
        print(f"x = {x:.2f} 1/min ist ein Minimum (f""(x) = {wert_zweite_ableitung_25grad:.4f})")
    elif wert_zweite_ableitung_25grad < 0:
        print(f"x = {x:.2f} 1/min ist ein Maximum (f""(x) = {wert_zweite_ableitung_25grad:.4f})")
    else:
        print(f"x = {x:.2f} 1/min könnte ein Sattelpunkt sein (f""(x) = {wert_zweite_ableitung_25grad:.4f})")

print(" ")
print("α = 30°:")
for x in nullstellen_30grad:
    wert_zweite_ableitung_30grad = zweite_ableitung_30grad(x)
    if wert_zweite_ableitung_30grad > 0:
        print(f"x = {x:.2f} 1/min ist ein Minimum (f""(x) = {wert_zweite_ableitung_30grad:.4f})")
    elif wert_zweite_ableitung_30grad < 0:
        print(f"x = {x:.2f} 1/min ist ein Maximum (f""(x) = {wert_zweite_ableitung_30grad:.4f})")
    else:
        print(f"x = {x:.2f} 1/min könnte ein Sattelpunkt sein (f""(x) = {wert_zweite_ableitung_30grad:.4f})")

print(" ")
print("α = 50°:")
for x in nullstellen_50grad:
    wert_zweite_ableitung_50grad = zweite_ableitung_50grad(x)
    if wert_zweite_ableitung_50grad > 0:
        print(f"x = {x:.2f} 1/min ist ein Minimum (f""(x) = {wert_zweite_ableitung_50grad:.4f})")
    elif wert_zweite_ableitung_50grad < 0:
        print(f"x = {x:.2f} 1/min ist ein Maximum (f""(x) = {wert_zweite_ableitung_50grad:.4f})")
    else:
        print(f"x = {x:.2f} 1/min könnte ein Sattelpunkt sein (f""(x) = {wert_zweite_ableitung_50grad:.4f})")

MPP_20grad=regression_kubisch_20grad(nullstellen_20grad[0])
MPP_25grad=regression_kubisch_25grad(nullstellen_25grad[0])
MPP_30grad=regression_kubisch_30grad(nullstellen_30grad[0])
MPP_50grad=regression_kubisch_50grad(nullstellen_50grad[0])
print(" ")
print("Der MPP mit α = 20° liegt bei", f"{MPP_20grad:.2f}", "mW")
print("Der MPP mit α = 25° liegt bei", f"{MPP_25grad:.2f}", "mW")
print("Der MPP mit α = 30° liegt bei", f"{MPP_30grad:.2f}", "mW")
print("Der MPP mit α = 50° liegt bei", f"{MPP_50grad:.2f}", "mW")

# Diagramme plotten

# Bereiche für glatte Regressionskurven (angepasst an tatsächliche Messbereiche)
reg_bereich_20grad = np.linspace(min(df_20grad["D [1/min]"]), max(df_20grad["D [1/min]"]), 100)
reg_bereich_25grad = np.linspace(min(df_25grad["D [1/min]"]), max(df_25grad["D [1/min]"]), 100)
reg_bereich_30grad = np.linspace(min(df_30grad["D [1/min]"]), max(df_30grad["D [1/min]"]), 100)
reg_bereich_50grad = np.linspace(min(df_50grad["D [1/min]"]), max(df_50grad["D [1/min]"]), 100)

# Vergleichsdiagramm erstellen
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Messdaten plotten
ax.plot(df_20grad["D [1/min]"], df_20grad["P [mW]"], "o", label="α = 20° Messdaten", color="blue")
ax.plot(df_25grad["D [1/min]"], df_25grad["P [mW]"], "s", label="α = 25° Messdaten", color="green")
ax.plot(df_30grad["D [1/min]"], df_30grad["P [mW]"], "^", label="α = 30° Messdaten", color="orange")
ax.plot(df_50grad["D [1/min]"], df_50grad["P [mW]"], "d", label="α = 50° Messdaten", color="red")

# Regressionskurven plotten
ax.plot(reg_bereich_20grad, reg_20grad(reg_bereich_20grad), "--", color="lightblue", label="Regression α = 20°")
ax.plot(reg_bereich_25grad, reg_25grad(reg_bereich_25grad), ":", color="lightgreen", label="Regression α = 25°")
ax.plot(reg_bereich_30grad, reg_30grad(reg_bereich_30grad), "-.", color="orange", alpha=0.7, label="Regression α = 30°")
ax.plot(reg_bereich_50grad, reg_50grad(reg_bereich_50grad), "-", color="pink", alpha=0.7, label="Regression α = 50°")

# MPP markieren
ax.plot(nullstellen_20grad[0], MPP_20grad, "o", label="MPP α = 20°", color="darkblue", markersize=8)
ax.plot(nullstellen_25grad[0], MPP_25grad, "s", label="MPP α = 25°", color="darkgreen", markersize=8)
ax.plot(nullstellen_30grad[0], MPP_30grad, "^", label="MPP α = 30°", color="darkorange", markersize=8)
ax.plot(nullstellen_50grad[0], MPP_50grad, "d", label="MPP α = 50°", color="darkred", markersize=8)

ax.set_xlabel("Drehzahl D [1/min]")
ax.set_ylabel("Leistung P [mW]")
ax.set_title("D-P-Diagramm: Vergleich aller Anstellwinkel (mit MPP)")
ax.grid()
ax.set_xlim(0)
ax.set_ylim(0)
ax.legend()

plt.tight_layout()
plt.show()

print("")

print("")
print("=" * 70)
print("ANALYSE UND BEWERTUNG:")
print("=" * 70)
print("TODO: Ergänzen Sie Ihre Analyse:")
print("1. Welcher Anstellwinkel zeigt die höchste MPP-Leistung?")
print("2. Wie verändert sich die optimale Drehzahl mit dem Anstellwinkel?")
print("3. Welche aerodynamischen Effekte erklären die Unterschiede?")
print("4. Wie kann das beobachtete Verhalten für den Betrieb realer Windkraftanlagen genutzt werden?")
print("5. Wie heißt die entsprechende Regelung bei Windkraftanlagen?")
print("=" * 70)

### Kritische Analyse der MPP-Berechnung

**TODO: Beschreiben Sie die Plausibilität der Ergebnissen**

---

### Auswertung und Diskussion der Ergebnisse

#### Anstellwinkel-Einfluss auf die Leistung

**TODO: Vervollständigen Sie basierend auf Ihren Messergebnissen**

**Messergebnis:** Der Anstellwinkel α = __° erreicht die höchste Maximalleistung von __ mW, während α = __° die niedrigste Leistung von __ mW zeigt.

**Warum beeinflusst der Anstellwinkel die Leistung so stark?**

TODO: Erklären Sie die aerodynamischen Grundlagen:
- Wie wirkt sich der Anstellwinkel auf Auftrieb und Widerstand aus?
- Warum gibt es einen optimalen Anstellwinkel?
- Was passiert bei zu kleinen bzw. zu großen Anstellwinkeln?

#### Praktische Anwendung für reale Windkraftanlagen

**Schutz vor Überlastung**
TODO: Erklären Sie, wie die Anstellwinkel-Verstellung bei Starkwind genutzt wird:
- Welche Regelung nutzt die Anstellwinkel-Verstellung?
- Wie wird die Anlage vor Schäden geschützt?
- Warum ist dies bei modernen Windkraftanlagen wichtig?

## 2.3 Versuch 3: Energiebilanz, Wirkungsgrad und Schnelllaufzahl einer Windkraftanlage
In diesem Versuch werden Energiebilanz, Wirkungsgrad und Schnelllaufzahl einer Windkraftanlage experimentell bestimmt. Ziel ist es, die erzeugte elektrische Leistung mit der Windleistung zu vergleichen, den Wirkungsgrad zu berechnen und die Schnelllaufzahl zu ermitteln.
Die Messungen erfolgen bei verschiedenen Netzspannungen mit einem AV-Modul, Windgeschwindigkeitsmessgerät und Drehzahlmessgerät.

---

**Aufbau, Geräte & Komponenten:**

- Grundeinheit  
- Winderzeugermodul  
- Power-Modul  
- Windgeneratormodul (3-Flügeler, 25°, optimiertes Profil)  
- Widerstandsmodul mit 100 Ω Steckelement  
- AV-Modul  
- Windgeschwindigkeitsmessgerät  
- Maßband  
- Drehzahlmessgerät  
- Thermometer

---

**Schaltpläne:**

Aufbau zur Ermittlung von Energiebilanz und Wirkungsgrad:

![Schaltplan Versuch3.1](Data/Schaltplan_Wind_Versuch_3-1.png "Energiebilanz & Wirkungsgrad")

Aufbau zur Ermittlung der Schnelllaufzahl:

![Schaltplan Versuch3.2](Data/Schaltplan_Wind_Versuch_3-2.png "Schnelllaufzahl")

---

**Durchführung**

1. Bauen Sie den Versuchsaufbau zur Ermittlung der Energiebilanz und des Wirkungsgrades auf.
   1. Stellen Sie eine Netzspannung von $U_\text{Netz} = 9\ \text{V}$ ein.
   2. Verwenden Sie zunächst das Windgeschwindigkeitsmessgerät anstelle des Windgenerators und messen Sie die Windgeschwindigkeit $v_\text{Wind}$.
   3. Setzen Sie den Windgenerator ein und ermitteln Sie Spannung $U$ und Stromstärke $I$ mit dem AV-Modul.
   4. Messen Sie den Rotordurchmesser $d$ und die Temperatur $\vartheta$.
2. Bauen Sie den zweiten Versuchsaufbau zur Bestimmung der Schnelllaufzahl auf.
   1. Stellen Sie eine Netzspannung von $U_\text{Netz} = 12\ \text{V}$ ein.
   2. Platzieren Sie das Windgeschwindigkeitsmessgerät und messen Sie die Windgeschwindigkeit $v_\text{Wind}$.
   3. Setzen Sie den Windgenerator ein und messen Sie die Drehzahl $D$.

---

**Messgrößen:**  

- Die Windgeschwindigkeit $v_\text{Wind}$ bei 9 V und 12 V in $\text{[m/s]}$  
- Die elektrische Spannung $U$ in $\text{[V]}$  
- Die Stromstärke $I$ in $\text{[mA]}$  
- Die Leistung $P = U \cdot I$ in $\text{[mW]}$  
- Die Temperatur $\vartheta$ in $\text{[°C]}$  
- Der Rotordurchmesser $d$ in $\text{[mm]}$  
- Die Drehzahl $D$ in $\text{[min}^{-1}\text{]}$  
- Die Luftdichte $\rho_\text{Luft}$ in $\text{[kg/m}^3\text{]}$ (aus Diagramm)  

---

**Messdaten:**

Notieren Sie hier Ihre Messdaten in einfachen Listen. Nutzen Sie dafür eindeutige Bezeichnungen, ähnlich wie im folgenden Beispiel:

In [ ]:
# DATENERFASSUNG - NUR MESSWERTE EINTRAGEN!

# =============================================================================
# TEIL 1: ENERGIEBILANZ UND WIRKUNGSGRAD (bei 9V Netzspannung)
# =============================================================================

# Windgeschwindigkeit bei 9V (vor Einbau des Generators)
v_9v = [5.01]  # Windgeschwindigkeit in m/s

# Elektrische Messgrößen bei 9V
u_9v = [3.06]       # Spannung in V
i_9v = [31.3]       # Stromstärke in mA

# Geometrische und Umgebungsdaten
d_rotor = [126.6]           # Rotordurchmesser in mm
temperatur = [21.6]         # Temperatur in °C

# =============================================================================
# TEIL 2: SCHNELLLAUFZAHL (bei 12V Netzspannung)
# =============================================================================

# Windgeschwindigkeit bei 12V
v_12v = [6.18]  # Windgeschwindigkeit in m/s

# Drehzahl bei 12V 
D_12v = [5764]  # Drehzahl in 1/min

print("Bereit für die Auswertung in den nächsten Zellen!")

Aus diesen Messwerten werden nun Tabellen mit `pandas` `DataFrames` erstellt.
Wichtige Hinweise:

- Die Messwerte müssen nicht nochmal eingegeben werden, solange die Zelle mit den Listen zuvor ausgeführt wurde
- Python beginnt die Indexierung bei [0] - der erste Wert hat Index 0, der zweite Index 1, usw.
- DataFrames ermöglichen professionelle Tabellendarstellung und einfache Berechnungen

Im folgenden Code werden alle Listen aus Messdaten in `DataFrames`umgewandelt:

In [ ]:
# DATAFRAMES AUS DEN LISTEN ERSTELLEN  

# Teil 1: Energiebilanz und Wirkungsgrad
df_energiebilanz = pd.DataFrame({
    "v_Wind [m/s]": ,
    "U [V]": ,
    "I [mA]": ,
    "d_Rotor [mm]": ,
    "Temperatur [°C]": 
})

# Teil 2: Schnelllaufzahl
df_schnelllaufzahl = pd.DataFrame({
    "v_Wind [m/s]": ,
    "D [1/min]": 
})

# Indizes als "Messpunkte" umbenennen (Zählung bei 1 starten)
for df in [df_energiebilanz, df_schnelllaufzahl]:
    df.index = [f"Messpunkt {i+1}" for i in range(len(df))]

print("DataFrames erfolgreich erstellt:")

In [ ]:
# MESSDATENTABELLEN ANZEIGEN

print("Tabelle 17: Messdaten zur Bestimmung der Energiebilanz und des Wirkungsgrads")
display(df_.round(2).style.format(precision=2))

print("\nTabelle 18: Messdaten zur Bestimmung der Schnelllaufzahl")
display(df_.round(2).style.format(precision=2))

### Beobachtungen während des Versuchs
Beschreiben Sie hier **in eigenen Worten** den Aufbau, die Durchführung und Ihre Beobachtungen:

- Wie unterschieden sich die Windgeschwindigkeiten bei 9V und 12V Netzspannung und welchen Einfluss hatte das auf die Messungen?
- Welche Unterschiede stellten Sie zwischen der theoretischen Windleistung und der tatsächlich erzeugten elektrischen Leistung fest?
- Was konnten Sie über den Wirkungsgrad der Windkraftanlage beobachten?
- Welche Schlüsse ziehen Sie aus der berechneten Schnelllaufzahl bezüglich der Rotorgeschwindigkeit?

> *Hinweis: Denken Sie an die physikalischen Grundlagen - warum kann nie die gesamte Windenergie umgewandelt werden?*

---

**Im folgenden Feld Ihre eigene Beschreibung und Beobachtungen eintragen:**

# Auswertung
1. Berechnen Sie aus den **9V-Messungen** die Angriffsfläche und die Windenergie vor dem Rotor $P_\text{Wind}$.  
   Ermitteln Sie die Dichte $\rho$ der Luft anhand des beigefügten Diagramms aus der gemessenen Temperatur.
2. Berechnen Sie aus den **9V-Messungen** die Nennleistung $P_N$ der Windkraftanlage.
3. Ermitteln Sie aus den **9V-Messungen** den Energiebilanzquotienten $\varepsilon$ und den Wirkungsgrad $\eta$ der Windkraftanlage.
4. Berechnen Sie aus den **12V-Messungen** die Schnelllaufzahl $\lambda$ der Windkraftanlage.

### Schritt 1: Angriffsfläche und Windenergie berechnen
**Grundlagen:**
Die **Angriffsfläche** $A$ entspricht der vom Rotor überstrichenen Kreisfläche und berechnet sich aus dem Durchmesser:
$$A = \frac{\pi \cdot d^2}{4}$$

Die **Windenergie**, die einem Rotor zur Verfügung steht, berechnet sich nach der Formel:
$$P_\text{Wind} = \frac{1}{2} \cdot \rho \cdot A \cdot v^3$$

Dabei ist:
- $\rho$ die Luftdichte in [kg/m³] (aus Diagramm bei gemessener Temperatur)
- $A$ die Angriffsfläche des Rotors in [m²] 
- $v$ die Windgeschwindigkeit in [m/s]
- $d$ der Rotordurchmesser in [m]

Die **Luftdichte** $\rho$ ist Abhängig voN Druck und Temperatur und kann für Umgebungsdruck (1,01325 bar) händisch aus dem dem folgenden Diagramm abgelesen:

![Luftdichte-Temperatur-Diagramm](Data/Luftdichte_Diagramm.png "Dichte der Luft in Abhängigkeit von der Raumtemperatur")

Die Windleistung steigt mit der **dritten Potenz** der Windgeschwindigkeit - eine Verdopplung der Windgeschwindigkeit führt zu einer achtfachen Windleistung!

**Was passiert im folgenden Code:**
- Aus der gemessenen Temperatur wird die Luftdichte $\rho$ über das Diagramm bestimmt
- Die Angriffsfläche $A$ wird aus dem Rotordurchmesser berechnet
- Die theoretisch verfügbare Windleistung $P_\text{Wind}$ wird für die **9V-Messungen** ermittelt

#### Alternative: Berechnung der Luftdichte mithilfe von CoolProp

CoolProp ist eine Open-Source-Bibliothek für die Berechnung thermodynamischer Stoffdaten von Fluiden. Die Bibliothek wird in der Industrie und Forschung weit verbreitet eingesetzt und enthält präzise Zustandsgleichungen für über 100 Fluide, einschließlich feuchter Luft.

**Vorteile von CoolProp**
- Berechnungen basierend auf wissenschaftlichen Zustandsgleichungen
- Berücksichtigung verschiedener Drücke und Luftfeuchtigkeit möglich

**Verwendung zur Berechnung der Luftdichte aus der gemessen Temperatur bei kosntantem Druck:**
```python
from CoolProp.CoolProp import PropsSI

# Luftdichte bei gegebener Temperatur und Standarddruck berechnen
t_kelvin = df_energiebilanz["Temperatur [°C]"] + 273.15  # Umrechnung nach Kelvin
druck = 101325  # Standarddruck in Pascal (1013.25 mbar)

# PropsSI(Output, Input1, Value1, Input2, Value2, Fluid)
rho_luft = PropsSI("D", "T", t_kelvin, "P", druck, "Air")
print(f"Luftdichte bei {temperatur_celsius}°C: {rho_luft:.3f} kg/m³")
```
---

#### Alternative: Berechnung der Luftdichte mithilfe von CoolProp

CoolProp ist eine Open-Source-Bibliothek für die Berechnung thermodynamischer Stoffdaten von Fluiden. Die Bibliothek wird in der Industrie und Forschung weit verbreitet eingesetzt und enthält präzise Zustandsgleichungen für über 100 Fluide, einschließlich feuchter Luft.

**Vorteile von CoolProp**
- Berechnungen basierend auf wissenschaftlichen Zustandsgleichungen
- Berücksichtigung verschiedener Drücke und Luftfeuchtigkeit möglich

**Verwendung zur Berechnung der Luftdichte aus der gemessen Temperatur bei kosntantem Druck:**
```python
from CoolProp.CoolProp import PropsSI

# Luftdichte bei gegebener Temperatur und Standarddruck berechnen
t_kelvin = df_energiebilanz["Temperatur [°C]"] + 273.15  # Umrechnung nach Kelvin
druck = 101325  # Standarddruck in Pascal (1013.25 mbar)

# PropsSI(Output, Input1, Value1, Input2, Value2, Fluid)
rho_luft = PropsSI("D", "T", t_kelvin, "P", druck, "Air")
print(f"Luftdichte bei df_energiebilanz["Temperatur [°C]"] °C: {rho_luft:.3f} kg/m³")
```
---

In [ ]:
# SCHRITT 1: ANGRIFFSFLÄCHE UND WINDENERGIE BERECHNEN

# Luftdichte aus Temperatur bestimmen (aus Diagramm ablesen)
df_energiebilanz["rho [kg/m³]"] = _  # Luftdichte aus Diagramm bei gemessener Temperatur

# Durchmesser von mm in m umrechnen
df_energiebilanz["d_Rotor [m]"] = df_energiebilanz["d_Rotor [mm]"] / _

# Angriffsfläche berechnen: A = π * d² / 4
df_energiebilanz["A_Rotor [m²]"] = np.pi * df_energiebilanz["_"]**2 / _

# Windleistung berechnen: P_Wind = 1/2 * ρ * A * v³
df_energiebilanz["P_Wind [W]"] = 0.5 * df_energiebilanz["_"] * df_energiebilanz["_"] * df_energiebilanz["_"]**_

# Ergebnis anzeigen
print("Berechnungen erfolgreich durchgeführt:")
display(df_energiebilanz.round(4).style.format(precision=4))

### Schritt 2: Nennleistung der Windkraftanlage berechnen
**Grundlagen:**
Die **Nennleistung** $P_N$ ist die tatsächlich vom Generator erzeugte elektrische Leistung:
$$P_N = U \cdot I$$

Diese wird aus den elektrischen Messgrößen der **9V-Messungen** berechnet:
- Spannung $U$ in [V] (Volt)
- Stromstärke $I$ in [mA] (Milliampere)
- Ergebnis: Nennleistung in [mW] (Milliwatt)

Die Nennleistung ist immer kleiner als die theoretisch verfügbare Windleistung, da Verluste durch Aerodynamik, Mechanik und Elektrik auftreten.

**Was passiert im folgenden Code:**
- Die elektrische Leistung wird aus Spannung und Stromstärke berechnet
- Das Ergebnis wird dem DataFrame hinzugefügt
- Die erweiterte Tabelle wird zur Kontrolle angezeigt

In [ ]:
# SCHRITT 2: NENNLEISTUNG BERECHNEN

# Nennleistung P_N = U × I berechnen
df_energiebilanz["P_N [mW]"] = df_energiebilanz["_"] * df_energiebilanz["_"]

# Nennleistung auch in Watt für Vergleich mit Windleistung
df_energiebilanz["P_N [W]"] = df_energiebilanz["P_N [mW]"] / _

print("Nennleistung erfolgreich berechnet:")
display(df_energiebilanz.round(4).style.format(precision=4))

### Schritt 3: Energiebilanzquotient und Wirkungsgrad ermitteln
**Grundlagen:**
Der **Energiebilanzquotient** $\varepsilon$ und der **Wirkungsgrad** $\eta$ beschreiben, wie effizient die Windkraftanlage arbeitet:
$$\varepsilon = \eta = \frac{P_N}{P_\text{Wind}}$$

Theoretisch kann nach dem **Betz"schen Gesetz** maximal 59,3% der Windenergie genutzt werden. In der Praxis liegen die Wirkungsgrade moderner Windkraftanlagen bei 35-45% aufgrund weiterer Verluste:
- Aerodynamische Verluste an den Rotorblättern
- Mechanische Verluste im Getriebe
- Elektrische Verluste im Generator und der Leistungselektronik

**Was passiert im folgenden Code:**
- Der Quotient aus Nennleistung und Windleistung wird berechnet
- Das Ergebnis wird als Dezimalzahl und in Prozent ausgegeben
- Eine Bewertung der Effizienz erfolgt im Vergleich zu theoretischen Werten

In [ ]:
# SCHRITT 3: ENERGIEBILANZQUOTIENT UND WIRKUNGSGRAD BERECHNEN

# Wirkungsgrad η = P_N / P_Wind berechnen
df_energiebilanz["Wirkungsgrad [-]"] = df_energiebilanz["_"] / df_energiebilanz["_"]

# Wirkungsgrad auch in Prozent
df_energiebilanz["Wirkungsgrad [%]"] = df_energiebilanz["Wirkungsgrad [-]"] * _

print("Wirkungsgrad erfolgreich berechnet:")
display(df_energiebilanz.round(4).style.format(precision=4))

# Bewertung des Wirkungsgrades
eta_prozent = df_energiebilanz["Wirkungsgrad [%]"].iloc[0]
print("Bewertung des Wirkungsgrades:")
print(f"Der gemessener Wirkungsgrad liegt bei {eta_prozent:.2f} %")
print("Das theoretische Limit nach Betz liegt bei 59.3 %")
print("Daraus lässt sich ableiten, dass...")

### Schritt 4: Schnelllaufzahl berechnen
**Grundlagen:**
Die **Schnelllaufzahl** $\lambda$ (Lambda) beschreibt das Verhältnis zwischen der Umfangsgeschwindigkeit der Rotorblätter und der Windgeschwindigkeit:
$$\lambda = \frac{v_\text{Blattspitze}}{v_\text{Wind}} = \frac{\omega \cdot r}{v_\text{Wind}}$$

Dabei ist die Umfangsgeschwindigkeit:
$$v_\text{Blattspitze} = \omega \cdot r = \frac{2\pi \cdot D}{60} \cdot \frac{d}{2}$$

Für die Berechnung werden die **12V-Messungen** verwendet:
- $D$ = Drehzahl in [min⁻¹] 
- $d$ = Rotordurchmesser in [m]
- $v_\text{Wind}$ = Windgeschwindigkeit in [m/s]

Moderne Windkraftanlagen arbeiten bei optimalen Schnelllaufzahlen zwischen 6-8. Zu niedrige Werte bedeuten schlechte Windausnutzung, zu hohe Werte führen zu erhöhtem Verschleiß und Lärm.

**Was passiert im folgenden Code:**
- Die Umfangsgeschwindigkeit der Blattspitzen wird berechnet
- Die Schnelllaufzahl wird als Verhältnis zur Windgeschwindigkeit der **12V-Messungen** ermittelt
- Eine Bewertung der Schnelllaufzahl erfolgt im Vergleich zu optimalen Betriebsbereichen

In [ ]:
# SCHRITT 4: SCHNELLLAUFZAHL BERECHNEN

# Umfangsgeschwindigkeit der Blattspitzen: v = ω * r = (2π * D / 60) * (d / 2)
# Rotordurchmesser aus 9V-Messung verwenden (in m)
d_rotor_m = df_energiebilanz["d_Rotor [m]"][0]

# Umfangsgeschwindigkeit berechnen
df_schnelllaufzahl["v_Blattspitze [m/s]"] = (2 * np.pi * df_schnelllaufzahl["_"] / _) * (d_rotor_m / _)

# Schnelllaufzahl λ = v_Blattspitze / v_Wind berechnen
df_schnelllaufzahl["Lambda [-]"] = df_schnelllaufzahl["_"] / df_schnelllaufzahl["_"]

print("Schnelllaufzahl erfolgreich berechnet:")
display(df_energiebilanz.round(4).style.format(precision=3))

# Bewertung der Schnelllaufzahl
lambda_wert = df_schnelllaufzahl["Lambda [-]"].iloc[0]
print(f"\nBewertung der Schnelllaufzahl:")
print(f"Gemessene Schnelllaufzahl: {lambda_wert:.2f}")
print("Schlussfolgerung:")

### Auswertung und Diskussion der Ergebnisse

#### Energiebilanz und Wirkungsgrad

**TODO: Vervollständigen Sie basierend auf Ihren Messergebnissen**

**Messergebnis:** Bei einer Windgeschwindigkeit von __ m/s steht eine theoretische Windleistung von __ W zur Verfügung. Die tatsächlich erzeugte elektrische Leistung beträgt __ mW, was einem Wirkungsgrad von __% entspricht.

**Warum kann nie die gesamte Windenergie genutzt werden?**

TODO: Erklären Sie die physikalischen Grundlagen:
- Was besagt das Betz'sche Gesetz und warum liegt das theoretische Maximum bei 59,3%?
- Welche zusätzlichen Verluste treten in realen Windkraftanlagen auf?
- Wie bewerten Sie Ihren gemessenen Wirkungsgrad im Vergleich zu modernen Anlagen (35-45%)?

#### Schnelllaufzahl und Rotorgeschwindigkeit

**Messergebnis:** Bei einer Windgeschwindigkeit von __ m/s und einer Drehzahl von __ min⁻¹ ergibt sich eine Schnelllaufzahl von λ = __.

**Bedeutung der Schnelllaufzahl für den Anlagenbetrieb**
TODO: Erklären Sie die praktische Relevanz:
- Liegt Ihre Schnelllaufzahl im optimalen Bereich (6-8) für Windkraftanlagen?
- Was sind die Auswirkungen von zu niedrigen bzw. zu hohen Schnelllaufzahlen?
- Warum ist die Schnelllaufzahl ein wichtiger Auslegungsparameter?

#### Praktische Anwendung für reale Windkraftanlagen

**Leistungsregelung und Effizienzoptimierung**
TODO: Erklären Sie, wie die Erkenntnisse in der Praxis genutzt werden:
- Wie werden moderne Windkraftanlagen geregelt, um den optimalen Wirkungsgrad zu erreichen?
- Welche Rolle spielt die Schnelllaufzahl bei der Anlagenauslegung?
- Warum ist die Energiebilanz wichtig für die Wirtschaftlichkeit von Windparks?


 
---

# 3. Quellen